In [3]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from global_vars import *
import torch

model_saved = "bert_mlm2023-03-05-18-13-19"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
model.load_state_dict(torch.load(model_saved, map_location=torch.device('cpu')))
model.eval();

In [6]:
from torch.nn import functional as F

input_text = "Given: All things in moderation  Sims is indeed a video game, and a good one at that Note a good one at that is somewhat subjective in that I am not completely promoting the death of any given Sim" + tokenizer.mask_token
input = tokenizer(input_text, return_tensors="pt")

mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)

output = model(**input)
logits = output.logits

softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_5 = torch.topk(mask_word, 2, dim = 1)[1][0]

for token in top_5:
    word = tokenizer.decode([token])
    new_sentence = input_text.replace(tokenizer.mask_token, word)
    print(new_sentence)

Given: All things in moderation  Sims is indeed a video game, and a good one at that Note a good one at that is somewhat subjective in that I am not completely promoting the death of any given Sim IN
Given: All things in moderation  Sims is indeed a video game, and a good one at that Note a good one at that is somewhat subjective in that I am not completely promoting the death of any given Sim INT
